In [3]:
path = "/home/efectn/python-music-classification"

In [4]:
import tensorflow as tf
import numpy as np

# 1. Modeli yükle
model = tf.keras.models.load_model(path + "/models/model_cnn.h5")


2025-05-23 21:17:25.460861: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-23 21:17:27.379610: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [25]:
import librosa
import math
import numpy as np

n_fft = 2048
hop_length = 512
n_mfcc = 13
segment_duration = 2 
target_frames = 88
genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']

audio, fs = librosa.load(path + "/blues.00002.wav", sr=22500)
duration = librosa.get_duration(y=audio, sr=fs)
print(f"Dosya süresi: {duration:.2f} saniye")

samples_per_segment = int(fs * segment_duration)

mid_point = len(audio) // 2
start_sample = max(0, mid_point - samples_per_segment // 2)
end_sample = start_sample + samples_per_segment
segment = audio[start_sample:end_sample]

if len(segment) < samples_per_segment:
    segment = np.pad(segment, (0, samples_per_segment - len(segment)))

mfcc = librosa.feature.mfcc(
    y=segment,
    sr=fs,
    n_fft=n_fft,
    hop_length=hop_length,
    n_mfcc=n_mfcc
).T

if mfcc.shape[0] < target_frames:
    pad_width = target_frames - mfcc.shape[0]
    mfcc = np.pad(mfcc, ((0, pad_width), (0, 0)))
elif mfcc.shape[0] > target_frames:
    mfcc = mfcc[:target_frames, :]

X_input = mfcc[..., np.newaxis]
X_input = np.expand_dims(X_input, axis=0)

def make_prediction(model, X):
    pred = model.predict(X, verbose=0)[0]
    pred_idx = np.argmax(pred)
    pred_genre = genres[pred_idx]
    avg_probs_percent = pred * 100
    return pred_genre, pred_idx, avg_probs_percent

genre_name, genre_index, avg_probs = make_prediction(model, X_input)

print(f"\nGenel Tahmin: {genre_name.upper()} (Sınıf ID: {genre_index})")
print("Ortalama Olasılıklar:")
for i, genre in enumerate(genres):
    print(f"  {genre:10s}: {avg_probs[i]:5.2f}%")


Dosya süresi: 30.01 saniye
Model girdi shape: (1, 88, 13, 1)

Genel Tahmin: BLUES (Sınıf ID: 0)
Ortalama Olasılıklar:
  blues     : 65.65%
  classical :  1.11%
  country   :  0.34%
  disco     : 28.24%
  hiphop    :  2.19%
  jazz      :  0.92%
  metal     :  1.01%
  pop       :  0.12%
  reggae    :  0.05%
  rock      :  0.36%
